In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#method 1 to get to 75% accuracy

application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [3]:
appcounts = application_df['APPLICATION_TYPE'].value_counts()
appcounts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
#trying to reduce the number of application types
application_types_to_replace = list(appcounts[appcounts<1000].index)
application_types_to_replace

['T8',
 'T7',
 'T10',
 'T9',
 'T13',
 'T12',
 'T2',
 'T25',
 'T14',
 'T29',
 'T15',
 'T17']

In [5]:
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
classificationcount = application_df['CLASSIFICATION'].value_counts()
classificationcount

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
classifications_to_replace = list(classificationcount[classificationcount<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
application_df = pd.get_dummies(application_df,dtype=float)

application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
y = application_df['IS_SUCCESSFUL'].values
y

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [10]:
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
X

array([[1.0000000e+00, 5.0000000e+03, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 1.0859000e+05, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 3.6500179e+07, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
number_features = len(X_train_scaled[0])
nodes_layer1=80
nodes_layer2=30
nodes_layer3=40

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=number_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=nodes_layer2,activation='relu'))

# Third hidden Layer

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3280      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


In [14]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# increase epochs to 200
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 0s 554us/step - loss: 0.5736 - accuracy: 0.7193
Epoch 2/200
804/804 [==============================] - 0s 585us/step - loss: 0.5582 - accuracy: 0.7281
Epoch 3/200
804/804 [==============================] - 0s 540us/step - loss: 0.5559 - accuracy: 0.7284
Epoch 4/200
804/804 [==============================] - 0s 539us/step - loss: 0.5542 - accuracy: 0.7289
Epoch 5/200
804/804 [==============================] - 0s 608us/step - loss: 0.5534 - accuracy: 0.7303
Epoch 6/200
804/804 [==============================] - 0s 569us/step - loss: 0.5517 - accuracy: 0.7318
Epoch 7/200
804/804 [==============================] - 0s 529us/step - loss: 0.5511 - accuracy: 0.7317
Epoch 8/200
804/804 [==============================] - 0s 537us/step - loss: 0.5506 - accuracy: 0.7315
Epoch 9/200
804/804 [==============================] - 0s 576us/step - loss: 0.5502 - accuracy: 0.7331
Epoch 10/200
804/804 [==============================] - 0s 584us/step - l

804/804 [==============================] - 0s 526us/step - loss: 0.5399 - accuracy: 0.7383
Epoch 80/200
804/804 [==============================] - 0s 550us/step - loss: 0.5396 - accuracy: 0.7384
Epoch 81/200
804/804 [==============================] - 0s 541us/step - loss: 0.5394 - accuracy: 0.7384
Epoch 82/200
804/804 [==============================] - 0s 513us/step - loss: 0.5389 - accuracy: 0.7388
Epoch 83/200
804/804 [==============================] - 0s 511us/step - loss: 0.5390 - accuracy: 0.7390
Epoch 84/200
804/804 [==============================] - 0s 533us/step - loss: 0.5392 - accuracy: 0.7391
Epoch 85/200
804/804 [==============================] - 0s 562us/step - loss: 0.5391 - accuracy: 0.7390
Epoch 86/200
804/804 [==============================] - 0s 526us/step - loss: 0.5391 - accuracy: 0.7385
Epoch 87/200
804/804 [==============================] - 0s 504us/step - loss: 0.5392 - accuracy: 0.7391
Epoch 88/200
804/804 [==============================] - 0s 511us/step - loss:

804/804 [==============================] - 0s 582us/step - loss: 0.5374 - accuracy: 0.7399
Epoch 158/200
804/804 [==============================] - 0s 579us/step - loss: 0.5366 - accuracy: 0.7400
Epoch 159/200
804/804 [==============================] - 0s 526us/step - loss: 0.5371 - accuracy: 0.7403
Epoch 160/200
804/804 [==============================] - 0s 497us/step - loss: 0.5367 - accuracy: 0.7403
Epoch 161/200
804/804 [==============================] - 0s 516us/step - loss: 0.5369 - accuracy: 0.7390
Epoch 162/200
804/804 [==============================] - 0s 564us/step - loss: 0.5366 - accuracy: 0.7402
Epoch 163/200
804/804 [==============================] - 0s 538us/step - loss: 0.5369 - accuracy: 0.7399
Epoch 164/200
804/804 [==============================] - 0s 535us/step - loss: 0.5366 - accuracy: 0.7402
Epoch 165/200
804/804 [==============================] - 0s 536us/step - loss: 0.5372 - accuracy: 0.7390
Epoch 166/200
804/804 [==============================] - 0s 571us/ste

In [16]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5702 - accuracy: 0.7300
Loss: 0.5702178478240967, Accuracy: 0.7300291657447815


In [17]:
### Method 2 of increasing accuracy
application_df2 = pd.read_csv("Resources/charity_data.csv")
application_df2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
application_df2 = application_df2.drop(columns = ['EIN', 'NAME'])
appcounts2 = application_df2['APPLICATION_TYPE'].value_counts()
appcounts2

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [19]:
application_types_to_replace2 = list(appcounts2[appcounts2<1200].index)
application_types_to_replace2

['T5',
 'T19',
 'T8',
 'T7',
 'T10',
 'T9',
 'T13',
 'T12',
 'T2',
 'T25',
 'T14',
 'T29',
 'T15',
 'T17']

In [20]:
for app in application_types_to_replace2:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
Other     4504
T4        1542
T6        1216
Name: APPLICATION_TYPE, dtype: int64

In [21]:
classificationcount2 = application_df2['CLASSIFICATION'].value_counts()
classificationcount2

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [22]:
classifications_to_replace2 = list(classificationcount2[classificationcount2<2000].index)

# Replace in dataframe
for cls in classifications_to_replace2:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [23]:
application_df2 = pd.get_dummies(application_df2,dtype=float)

application_df2.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
y = application_df2['IS_SUCCESSFUL'].values
y

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [25]:
X = application_df2.drop('IS_SUCCESSFUL', axis=1).values
X

array([[1.0000000e+00, 5.0000000e+03, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 1.0859000e+05, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 3.6500179e+07, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
number_features = len(X_train_scaled[0])
nodes_layer1=80
nodes_layer2=30
nodes_layer3=40

nn2 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=number_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=nodes_layer2,activation='relu'))

# Third hidden Layer

# Output layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                2960      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 5,421
Trainable params: 5,421
Non-trainable params: 0
_________________________________________________________________


In [29]:
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
#again using 50 epochs
fit_model = nn2.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 0s 616us/step - loss: 0.5888 - accuracy: 0.7088
Epoch 2/50
804/804 [==============================] - 0s 584us/step - loss: 0.5734 - accuracy: 0.7214
Epoch 3/50
804/804 [==============================] - 0s 555us/step - loss: 0.5713 - accuracy: 0.7220
Epoch 4/50
804/804 [==============================] - 0s 509us/step - loss: 0.5682 - accuracy: 0.7233
Epoch 5/50
804/804 [==============================] - 0s 525us/step - loss: 0.5676 - accuracy: 0.7233
Epoch 6/50
804/804 [==============================] - 0s 577us/step - loss: 0.5660 - accuracy: 0.72350s - loss: 0.5677 - accura
Epoch 7/50
804/804 [==============================] - 0s 576us/step - loss: 0.5651 - accuracy: 0.7247
Epoch 8/50
804/804 [==============================] - 0s 554us/step - loss: 0.5651 - accuracy: 0.7254
Epoch 9/50
804/804 [==============================] - 0s 560us/step - loss: 0.5651 - accuracy: 0.7261
Epoch 10/50
804/804 [==============================] - 0

In [31]:
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5666 - accuracy: 0.7199
Loss: 0.5665838718414307, Accuracy: 0.719883382320404


In [32]:
###Method #3 adding the third nodes layer

number_features = len(X_train_scaled[0])
nodes_layer1=80
nodes_layer2=30
nodes_layer3=40

nn3 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn3.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=number_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn3.add(tf.keras.layers.Dense(units=nodes_layer2,activation='relu'))

# Third hidden Layer
nn3.add(tf.keras.layers.Dense(units=nodes_layer3,activation='relu'))
# Output layer
#  YOUR CODE GOES HERE
nn3.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 80)                2960      
_________________________________________________________________
dense_7 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_8 (Dense)              (None, 40)                1240      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 41        
Total params: 6,671
Trainable params: 6,671
Non-trainable params: 0
_________________________________________________________________


In [33]:
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 0s 568us/step - loss: 0.5855 - accuracy: 0.7109
Epoch 2/100
804/804 [==============================] - 0s 609us/step - loss: 0.5697 - accuracy: 0.7231
Epoch 3/100
804/804 [==============================] - 0s 608us/step - loss: 0.5682 - accuracy: 0.7227
Epoch 4/100
804/804 [==============================] - 0s 598us/step - loss: 0.5656 - accuracy: 0.7257
Epoch 5/100
804/804 [==============================] - 0s 593us/step - loss: 0.5656 - accuracy: 0.7244
Epoch 6/100
804/804 [==============================] - 1s 666us/step - loss: 0.5647 - accuracy: 0.7263
Epoch 7/100
804/804 [==============================] - 1s 652us/step - loss: 0.5635 - accuracy: 0.7256
Epoch 8/100
804/804 [==============================] - 0s 604us/step - loss: 0.5631 - accuracy: 0.7263
Epoch 9/100
804/804 [==============================] - 0s 585us/step - loss: 0.5629 - accuracy: 0.7257
Epoch 10/100
804/804 [==============================] - 1s 629us/step - l

804/804 [==============================] - 0s 606us/step - loss: 0.5551 - accuracy: 0.7310
Epoch 80/100
804/804 [==============================] - 0s 612us/step - loss: 0.5546 - accuracy: 0.7316
Epoch 81/100
804/804 [==============================] - 0s 558us/step - loss: 0.5547 - accuracy: 0.7311
Epoch 82/100
804/804 [==============================] - 0s 550us/step - loss: 0.5546 - accuracy: 0.7315
Epoch 83/100
804/804 [==============================] - 0s 595us/step - loss: 0.5547 - accuracy: 0.7315
Epoch 84/100
804/804 [==============================] - 0s 595us/step - loss: 0.5543 - accuracy: 0.7320
Epoch 85/100
804/804 [==============================] - 0s 587us/step - loss: 0.5546 - accuracy: 0.7314
Epoch 86/100
804/804 [==============================] - 0s 553us/step - loss: 0.5543 - accuracy: 0.7315
Epoch 87/100
804/804 [==============================] - 0s 555us/step - loss: 0.5544 - accuracy: 0.7313
Epoch 88/100
804/804 [==============================] - 0s 603us/step - loss:

In [35]:
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5705 - accuracy: 0.7198
Loss: 0.570472002029419, Accuracy: 0.7197667360305786


In [36]:
application_df3 = pd.read_csv("Resources/charity_data.csv")
application_df3.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [38]:
application_df3 = application_df3.drop(columns = ['EIN', 'NAME'])
appcounts3 = application_df3['APPLICATION_TYPE'].value_counts()
application_types_to_replace3 = list(appcounts3[appcounts3<500].index)
for app in application_types_to_replace3:
    application_df3['APPLICATION_TYPE'] = application_df3['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df3['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [43]:
classificationcount3 = application_df3['CLASSIFICATION'].value_counts()
classifications_to_replace3 = list(classificationcount3[classificationcount3<100].index)

# Replace in dataframe
for cls in classifications_to_replace3:
    application_df3['CLASSIFICATION'] = application_df3['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df3['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [44]:
application_df3 = pd.get_dummies(application_df3,dtype=float)

application_df3.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [54]:
y = application_df3['IS_SUCCESSFUL'].values
X = application_df3.drop('IS_SUCCESSFUL', axis=1).values

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [56]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [57]:
number_features = len(X_train_scaled[0])
nodes_layer1=80
nodes_layer2=30
nodes_layer3=15

nn4 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn4.add(tf.keras.layers.Dense(units=nodes_layer1, input_dim=number_features, activation='relu'))
# Second hidden layer
#  YOUR CODE GOES HERE
nn4.add(tf.keras.layers.Dense(units=nodes_layer2,activation='relu'))

# Third hidden Layer
nn4.add(tf.keras.layers.Dense(units=nodes_layer3,activation='relu'))
# Output layer
#  YOUR CODE GOES HERE
nn4.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
# Check the structure of the model
nn4.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 80)                3600      
_________________________________________________________________
dense_19 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_20 (Dense)             (None, 15)                465       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 16        
Total params: 6,511
Trainable params: 6,511
Non-trainable params: 0
_________________________________________________________________


In [58]:
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
fit_model = nn4.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 624us/step - loss: 0.5692 - accuracy: 0.7186
Epoch 2/100
804/804 [==============================] - 0s 592us/step - loss: 0.5533 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 0s 575us/step - loss: 0.5502 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 0s 595us/step - loss: 0.5490 - accuracy: 0.7306
Epoch 5/100
804/804 [==============================] - 1s 648us/step - loss: 0.5482 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 0s 599us/step - loss: 0.5473 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 0s 578us/step - loss: 0.5464 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 1s 630us/step - loss: 0.5461 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 1s 640us/step - loss: 0.5453 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 0s 611us/step - l

804/804 [==============================] - 0s 550us/step - loss: 0.5337 - accuracy: 0.7402
Epoch 80/100
804/804 [==============================] - 0s 536us/step - loss: 0.5344 - accuracy: 0.7393
Epoch 81/100
804/804 [==============================] - 0s 538us/step - loss: 0.5341 - accuracy: 0.7390
Epoch 82/100
804/804 [==============================] - 0s 556us/step - loss: 0.5342 - accuracy: 0.7396
Epoch 83/100
804/804 [==============================] - 0s 621us/step - loss: 0.5344 - accuracy: 0.7390
Epoch 84/100
804/804 [==============================] - 1s 650us/step - loss: 0.5339 - accuracy: 0.7389
Epoch 85/100
804/804 [==============================] - 1s 646us/step - loss: 0.5343 - accuracy: 0.7393
Epoch 86/100
804/804 [==============================] - 1s 691us/step - loss: 0.5341 - accuracy: 0.7391
Epoch 87/100
804/804 [==============================] - 1s 661us/step - loss: 0.5339 - accuracy: 0.7399
Epoch 88/100
804/804 [==============================] - 1s 700us/step - loss:

In [60]:
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5619 - accuracy: 0.7287
Loss: 0.5619065165519714, Accuracy: 0.7287463545799255


In [61]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE



nn.save("AlphabetSoupCharity_Optimization.h5")